# UFGsim2024infufg
> Module to handle loading, preprocessing and postprocessing of the data from the simulated world for UFG's dataset

In [69]:
#| default_exp UFGsim2024infufg

In [70]:
#| exports
import torch
import re
from torch.utils.data import Dataset, DataLoader, random_split
from torch import nn
import yaml
from pathlib import Path
import numpy as np
from lightning import LightningDataModule
from torchvision.transforms import v2

introduction here

In [71]:
#| exports
class UFGSimDataset(Dataset):
    "Load the UFGSim dataset ina pytorch Dataset object."
    def __init__(self, data_path, is_train=True, transform=None):
        data_path = Path(data_path)
        self.ufgsim_velodyne_path = data_path/'laser_scans'

        ufgsim_velodyne_fns = []

        ufgsim_velodyne_fns += list(self.ufgsim_velodyne_path.rglob('*laser[0-9]/*.bin'))

        self.frame_ids = [fn.stem for fn in sorted(ufgsim_velodyne_fns)]
        self.frame_lasers = [fn.parts[-2] for fn in ufgsim_velodyne_fns]
        
        self.transform = transform

    def __len(self):
        return len(self.frame_ids)
    
    def __getitem__(self, idx):
        frame_id = self.frame_ids[idx]
        frame_laser = self.frame_lasers[idx]

        frame_path = self.ufgsim_velodyne_path/frame_laser/(frame_id + '.bin')

        with open(frame_path, 'rb') as f:
            frame = np.fromfile(f, dtype=np.float32).reshape(-1, 4)
            x_frame = frame[:, 0]
            y_frame = frame[:, 1]
            z_frame = frame[:, 2]
            label = frame[:, 3].astype(np.uint8)

        return x_frame, y_frame, z_frame, label
        
        


instructions of download and label description

In [72]:
#| eval: false
data_path = 'A:/UFGSim'
ds = UFGSimDataset(data_path)
ds[127]

(array([-12.600517, -16.87647 , -25.433857, ..., -16.878117, -25.434309,
        -51.473537], dtype=float32),
 array([-3.3436601e-05, -4.4783228e-05, -6.7491019e-05, ...,
         4.4787597e-05,  6.7492227e-05,  1.3658965e-04], dtype=float32),
 array([-1.5471467, -1.4764977, -1.33293  , ..., -1.4766417, -1.3329537,
        -0.8984726], dtype=float32),
 array([4, 4, 4, ..., 4, 4, 4], dtype=uint8))

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()